In [1]:
# -*- encoding:utf-8 -*-

from __future__ import print_function

import tensorflow as tf
import keras
import os
import cPickle as pkl
from collections import Counter

from nltk import sent_tokenize, word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from stanfordcorenlp import StanfordCoreNLP
import jieba
jieba.enable_parallel(8)

import sys
stdout = sys.stdout
reload(sys)
sys.stdout = stdout
sys.setdefaultencoding('utf8')

stopwords = stopwords.words('english')
english_punctuations = [',','.',':',';','?','(',')','[',']','&','!','*','@','#','$','%']
stopwords += english_punctuations

# nlp = StanfordCoreNLP(r'/home/linxuming/nltk_data/stanford/stanford-corenlp-full-2016-10-31/', memory='8g')


raw_data_path = 'data/UM-Corpus/data'

/usr/share/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.309 seconds.
Prefix dict has been built succesfully.


In [4]:
def data_preprocess():
	train_en_corpus = []
	train_ch_corpus = []
	test_en_corpus = []
	test_ch_corpus = []
	count = 0

	for dirname in ('Bilingual', 'Testing'):
		if dirname == 'Bilingual':
			sub_dir_names = ['Education', 'Laws', 'Microblog', 'News', 
								'Science', 'Spoken', 'Subtitles', 'Thesis']
			for filename in sub_dir_names:
				with open(os.path.join(raw_data_path, dirname, filename, ''.join(['Bi-', filename, '.txt']))) as fr:
					for i, line in enumerate(fr):
						line = line.strip().decode()
						count += 1
						if i % 2 == 0:
							train_en_corpus.append(line)
						else:
							train_ch_corpus.append(line)
					print('Finished {}'.format(count))
		else:
			with open(os.path.join(raw_data_path, dirname, 'Testing-Data.txt')) as fr:
				for i, line in enumerate(fr):
					line = line.strip().decode()
					count += 1
					if i % 2 == 0:
						test_en_corpus.append(line)
					else:
						test_ch_corpus.append(line)
				print('Finished {}'.format(count))
	train_en_corpus = '\n'.join(train_en_corpus)
	train_ch_corpus = '\n'.join(train_ch_corpus)
	test_en_corpus = '\n'.join(test_en_corpus)
	test_ch_corpus = '\n'.join(test_ch_corpus)
	return train_en_corpus, train_ch_corpus, test_en_corpus, test_ch_corpus


def segment(corpus, tokenizer, savepath=None):
	tokenized_corpus = []
	count = 0
	tokenized_corpus = ' '.join([_ for _ in tokenizer(corpus) if _.strip(' ')])
	tokenized_corpus = tokenized_corpus.split(' \n ')
	# for sentence in corpus:
	# 	count += 1
	# 	tokenized_corpus.append(' '.join(tokenizer(sentence)))
	# 	if count % 1000 == 0:
	# 		print('Finished cutting {}'.format(count))
	if savepath:
		with open(savepath, 'w') as fw:
			pkl.dump(tokenized_corpus, fw)
	return tokenized_corpus



In [ ]:

train_en_corpus, train_ch_corpus, test_en_corpus, test_ch_corpus = data_preprocess()
train_en_corpus = segment(train_en_corpus, jieba.cut, 'data/preprocess/train_en_segment.pkl')
train_ch_corpus = segment(train_ch_corpus, lambda k: iter(k.strip()), 'data/preprocess/train_ch_segment.pkl')
test_en_corpus = segment(test_en_corpus, jieba.cut, 'data/preprocess/test_en_segment.pkl')
test_ch_corpus = segment(test_ch_corpus, lambda k: iter(k.strip()), 'data/preprocess/test_ch_segment.pkl')

In [5]:
with open('data/preprocess/train_en_segment.pkl') as fr:
    train_en_corpus = pkl.load(fr)

with open('data/preprocess/train_ch_segment.pkl') as fr:
    train_ch_corpus = pkl.load(fr)

with open('data/preprocess/test_en_segment.pkl') as fr:
    test_en_corpus = pkl.load(fr)

with open('data/preprocess/test_ch_segment.pkl') as fr:
    test_ch_corpus = pkl.load(fr)

def vocab(data, topK=None):
    word2id = Counter()
    for sentence in data:
        for word in sentence.split():
            word2id[word] += 1
    word2id = word2id.most_common()
    if topK:
        word2id = word2id[:topK]
    word2id, _ = zip(*word2id)
    word2id = {word : i + 4 for i, word in enumerate(word2id)}
    word2id['<PAD>'] = 0
    word2id['<UNK>'] = 1
    word2id['<S>'] = 2
    word2id['</S>'] = 3
    id2word = dict(zip(word2id.values(), word2id.keys()))
    return word2id, id2word

en_word2id, en_id2word = vocab(train_en_corpus)
ch_word2id, ch_id2word = vocab(train_ch_corpus)

In [6]:
from sklearn.model_selection import train_test_split

print(len(en_word2id))
print(len(ch_word2id))

max_src_len = 40
max_tgt_len = 40
src_embedding_size = 200
tgt_embedding_size = 200
train_size = 0.8


def transform(data, word2id):
    ret_data = []
    for sentence in data:
        ret_data.append([word2id.get(word, 1) for word in sentence.split()])
    return ret_data


train_en_corpus = transform(train_en_corpus, en_word2id)
train_ch_corpus = transform(train_ch_corpus, ch_word2id)
with open('data/preprocess/vocab_dict.pkl', 'w') as fw:
    pkl.dump([en_word2id, en_id2word, ch_word2id, ch_id2word], fw)
with open('data/preprocess/vocab_dict_and_corpus.pkl', 'w') as fw:
    pkl.dump([en_word2id, en_id2word, ch_word2id, ch_id2word, train_en_corpus, train_ch_corpus, test_en_corpus, test_ch_corpus], fw)

with open('data/preprocess/demo_vocab_dict_and_corpus.pkl', 'w') as fw:
    pkl.dump([en_word2id, en_id2word, ch_word2id, ch_id2word, train_en_corpus[:100], train_ch_corpus[:100], test_en_corpus[:100], test_ch_corpus[:100]], fw)


537922
9893


In [ ]:
def padding(data, max_len):
    return tf.keras.preprocessing.sequence.pad_sequences(data, max_len, padding='post', truncating='post')

with open('data/preprocess/vocab_dict.pkl') as fr:
    en_word2id, en_id2word, ch_word2id, ch_id2word, \
    train_en_corpus, train_ch_corpus, test_en_corpus, test_ch_corpus = pkl.load(fr)

    
train_en_corpus = padding(train_en_corpus, max_src_len)
train_ch_corpus = padding(train_ch_corpus, max_tgt_len)

print(train_en_corpus.shape)
print(train_ch_corpus.shape)

In [ ]:
max_src_vocab_size = 60000
max_tgt_vocab_size = 5000

print(train_en_corpus[10])
print([en_id2word[_] for _ in train_en_corpus[10]])
print(' '.join([ch_id2word[_] for _ in train_ch_corpus[10]]))
# train_en_corpus[train_en_corpus >= max_src_vocab_size] = 1
# train_ch_corpus[train_ch_corpus >= max_tgt_vocab_size] = 1